## Instruction
In this notebook, We use data which I donwload from Tushare to make up some alpha factors and risk factors.
1. Load data from Tushare time range bettwen 2017.1 - 2023.3.
2. Calculate portfolio risk by PCA and save idiosynchritic values view a factor
3. Make up some factors like we did in P4 project.
5. Evaluate factor returns in 5D, 20D, 60D, 120D. and seperate factors into two parts, risk factors and alpha factors.

## Load Data

In [1]:
import pandas as pd 
import numpy as np
from tqdm import tqdm

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [6, 4]

import warnings
warnings.filterwarnings('ignore')

In [2]:
universe_raw = pd.read_csv('raw_20170103_20230317.csv').iloc[:,1:]
fundamental_df = pd.read_csv('fundamental_20170103_20230317.csv').iloc[:,1:]

## Data Explain
  - 'ts_code','ann_date','cfps','revenue_ps', 'quick_ratio',  每股现金流，每股营业收入，速冻比率
  - 'dt_eps','basic_eps_yoy','dt_eps_yoy',  每股收益
  - 'bps','bps_yoy',  每股净资产
  - 'extra_item','profit_dedt', 扣非，扣非净利润，扣费净利润单季度 
  - 'roe_dt','q_dt_roe','roe_yoy',  净资产收益
  - 'capital_rese_ps','surplus_rese_ps',  每股资本公积，每股公积盈余 
  - 'gross_margin','interestdebt','ca_to_assets', 毛利，速冻比率，带息债务
  - 'ebt_yoy','roe_yoy','or_yoy','equity_yoy' 总利润增长，净资产收益增长，营业收入增长，净资产增长

In [4]:
print(universe_raw.shape, len(fundamental_df.ts_code.unique()))

(867437, 57) 746


In [7]:
stock_list = np.random.choice(fundamental_df.ts_code.values, 200)
universe = universe_raw.loc[(universe_raw.ts_code.isin(stock_list)) \
                 & (universe_raw['trade_date']>=20180101) & (universe_raw['trade_date']<=20190101)]

In [13]:
universe = universe.append(universe_raw.loc[(universe_raw.ts_code=='603538.SH') \
                 & (universe_raw['trade_date']>=20180101) & (universe_raw['trade_date']<=20190101)])

In [14]:
universe.to_csv('tmp.csv')

In [15]:
universe.loc[universe.ts_code=='603538.SH']

,ts_code,trade_date,turnover_rate,volume_ratio,pe,pb,total_share,free_share,total_mv,circ_mv,...,roe_yoy,capital_rese_ps,surplus_rese_ps,gross_margin,interestdebt,ca_to_assets,ebt_yoy,roe_yoy.1,or_yoy,equity_yoy
759575,603538.SH,20180102,1.7829,0.77,41.3072,2.9414,12000.0,3000.0,325680.000,81420.00,...,-9.3402,4.2247,0.0868,1.449833e+08,1.400000e+08,56.3840,-43.1507,-9.3402,-4.0642,12.9135
759576,603538.SH,20180103,1.6366,0.74,41.3833,2.9468,12000.0,3000.0,326280.000,81570.00,...,-9.3402,4.2247,0.0868,1.449833e+08,1.400000e+08,56.3840,-43.1507,-9.3402,-4.0642,12.9135
759577,603538.SH,20180104,1.5866,0.84,40.9723,2.9176,12000.0,3000.0,323040.000,80760.00,...,-9.3402,4.2247,0.0868,1.449833e+08,1.400000e+08,56.3840,-43.1507,-9.3402,-4.0642,12.9135
759578,603538.SH,20180105,1.2232,0.71,41.0028,2.9197,12000.0,3000.0,323280.000,80820.00,...,-9.3402,4.2247,0.0868,1.449833e+08,1.400000e+08,56.3840,-43.1507,-9.3402,-4.0642,12.9135
759579,603538.SH,20180108,1.9092,1.13,40.8658,2.9100,12000.0,3000.0,322200.000,80550.00,...,-9.3402,4.2247,0.0868,1.449833e+08,1.400000e+08,56.3840,-43.1507,-9.3402,-4.0642,12.9135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759740,603538.SH,20181224,1.1784,0.77,66.2597,2.5580,14913.4,7918.0,296030.990,177578.10,...,73.0871,3.4663,0.0699,1.655237e+08,5.783932e+08,47.8834,69.2082,73.0871,30.0905,4.5195
759741,603538.SH,20181225,1.3669,0.84,66.3932,2.5632,14913.4,7918.0,296627.526,177935.94,...,73.0871,3.4663,0.0699,1.655237e+08,5.783932e+08,47.8834,69.2082,73.0871,30.0905,4.5195
759742,603538.SH,20181226,0.8985,0.61,65.9593,2.5464,14913.4,7918.0,294688.784,176772.96,...,73.0871,3.4663,0.0699,1.655237e+08,5.783932e+08,47.8834,69.2082,73.0871,30.0905,4.5195
759743,603538.SH,20181227,1.0501,0.75,64.6908,2.4975,14913.4,7918.0,289021.692,173373.48,...,73.0871,3.4663,0.0699,1.655237e+08,5.783932e+08,47.8834,69.2082,73.0871,30.0905,4.5195


In [16]:
universe.columns

Index(['ts_code', 'trade_date', 'turnover_rate', 'volume_ratio', 'pe', 'pb',
       'total_share', 'free_share', 'total_mv', 'circ_mv', 'name', 'industry',
       'list_date', 'amount', 'adj_factor', 'open', 'close', 'high', 'low',
       'macd', 'rsi_6', 'rsi_12', 'rsi_24', 'boll_upper', 'boll_mid',
       'boll_lower', 'cci', 'date', 'pct_change', 'type', 'p_change_min',
       'p_change_max', 'issue_price', 'issue_amount', 'type_value', 'cfps',
       'revenue_ps', 'quick_ratio', 'dt_eps', 'basic_eps_yoy', 'dt_eps_yoy',
       'bps', 'bps_yoy', 'extra_item', 'profit_dedt', 'roe_dt', 'q_dt_roe',
       'roe_yoy', 'capital_rese_ps', 'surplus_rese_ps', 'gross_margin',
       'interestdebt', 'ca_to_assets', 'ebt_yoy', 'roe_yoy.1', 'or_yoy',
       'equity_yoy'],
      dtype='object')